***IMPORT DES DONNEES DE TRAFIC, BRUIT, METEO ET SONDAGE***

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\PPBE_BdxMet\src\Carac_situ_acoustiq')
sys.path.append(r'C:\Users\martin.schoreisz\git\PPBE_BdxMet\src')

import os 
from datetime import datetime
import pandas as pd
import Connexion_Transfert as ct
from Import_stockage_donnees.Import_export_bdd import FichierMesureBruitCsv, transfertFichierMesure2Bdd

# DONNEES BRUIT

## Lecture des fichiers ods

In [4]:
# exemple de lecture 
test = FichierMesureBruitCsv(r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono1_10RuePierreRonsard\sono1_02-04-2022.csv')
# test.dfNiveauSpectre.head(10)

In [ ]:
# transfert vers Bdd par dossier
dossierSrc = r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono4_26RueFrancoisVillon'
transfertFichierMesure2Bdd(dossierSrc)